In [1]:
import numpy as np

from qonnx.core.datatype import DataType
from qonnx.transformation.infer_datatypes import InferDataTypes

from qonnx.util.cleanup import cleanup_model
from qonnx.util.inference_cost import inference_cost

from qonnx.transformation.pruning import PruneChannels

from finn.util.visualization import showInNetron
from qonnx.core.modelwrapper import ModelWrapper

In [2]:
model_file = './MY_MBLNET_V2_RESNET_classifier__best_mean_F1__BIPOLAR_Out__QONNX.onnx'

# Analyze layers to prune

In [3]:
model = ModelWrapper(model_file)
model = cleanup_model(model, True)

In [4]:
all_inits_names = [init.name for init in model.graph.initializer]

print(f'Number of initializers = {len(all_inits_names)}')

Number of initializers = 376


In [5]:
eps = 1e-10

layers_to_prune = {}

for idx, init_name in enumerate(all_inits_names):
    if "Quant" in init_name and "param1" in init_name:
    # It is a scale value, check it
        np_init = model.get_initializer(init_name)
        np_abs_val = np.abs(np_init)
        zero_idx = (np_abs_val < eps) * (np_abs_val > 0)
        if np.all(zero_idx == False):
            print(f'Index = {idx}. {init_name} was not appended, as there were no values under epsilon')
        else:
            zero_layer = np.where(zero_idx == True)[0]
            quant_layer_out_name = init_name.split("param")[0] + "out0"
            layers_to_prune[quant_layer_out_name] = {1: {*zero_layer}}
            print(f'Index = {idx}. {init_name} appended, as there were values under epsilon')

Index = 132. Quant_0_param1 appended, as there were values under epsilon
Index = 135. Quant_1_param1 appended, as there were values under epsilon
Index = 138. Quant_2_param1 was not appended, as there were no values under epsilon
Index = 141. Quant_3_param1 was not appended, as there were no values under epsilon
Index = 144. Quant_4_param1 was not appended, as there were no values under epsilon
Index = 147. Quant_5_param1 was not appended, as there were no values under epsilon
Index = 150. Quant_6_param1 appended, as there were values under epsilon
Index = 153. Quant_7_param1 appended, as there were values under epsilon
Index = 156. Quant_8_param1 was not appended, as there were no values under epsilon
Index = 159. Quant_9_param1 was not appended, as there were no values under epsilon
Index = 162. Quant_10_param1 was not appended, as there were no values under epsilon
Index = 165. Quant_11_param1 was not appended, as there were no values under epsilon
Index = 168. Quant_12_param1 appen

In [6]:
print(f'Number of layers to prune: {len(layers_to_prune)}')
for k, v in layers_to_prune.items():
    print(k, v)

Number of layers to prune: 19
Quant_0_out0 {1: {17}}
Quant_1_out0 {1: {17}}
Quant_6_out0 {1: {31}}
Quant_7_out0 {1: {31}}
Quant_12_out0 {1: {36, 6, 40, 9, 43, 12, 46, 17, 26, 28}}
Quant_13_out0 {1: {36, 6, 40, 9, 43, 12, 46, 17, 26, 28}}
Quant_15_out0 {1: {0, 65, 2, 3, 67, 75, 14, 81, 18, 50, 52, 93, 22, 23, 88, 92, 61}}
Quant_16_out0 {1: {0, 65, 2, 3, 67, 75, 14, 81, 18, 50, 52, 93, 22, 23, 88, 92, 61}}
Quant_18_out0 {1: {3, 6, 8, 9, 11, 16, 17, 18, 25, 26, 33, 37, 39, 42, 52, 54, 62, 63, 71, 72, 74, 82, 84, 86, 90, 92, 93, 94, 99, 100, 102, 107, 111, 113, 116, 123, 124, 125, 126}}
Quant_19_out0 {1: {3, 6, 8, 9, 11, 16, 17, 18, 25, 26, 33, 37, 39, 42, 52, 54, 62, 63, 71, 72, 74, 82, 84, 86, 90, 92, 93, 94, 99, 100, 102, 107, 111, 113, 116, 123, 124, 125, 126}}
Quant_21_out0 {1: {0, 4, 5, 7, 13, 16, 17, 20, 21, 27, 30, 31, 33, 34, 35, 37, 40, 41, 49, 53, 60, 61, 62, 63, 64, 68, 71, 72, 76, 78, 79, 80, 82, 85, 88, 90, 91, 93, 95, 100, 102, 103, 105, 106, 107, 108, 116, 117, 120, 121, 12

In [7]:
layers_to_prune.popitem()

('Quant_30_out0',
 {1: {0,
   2,
   6,
   7,
   8,
   10,
   11,
   13,
   14,
   17,
   20,
   21,
   26,
   30,
   32,
   33,
   35,
   42,
   43,
   47,
   49,
   50,
   52,
   55,
   58,
   61,
   63,
   65,
   66,
   67,
   68,
   69,
   70,
   73,
   78,
   79,
   85,
   87,
   88,
   90,
   91,
   95,
   98,
   100,
   101,
   102,
   103,
   104,
   106,
   107,
   109,
   110,
   112,
   113,
   114,
   117,
   119}})

# Prune the Layers

In [8]:
model = ModelWrapper(model_file)

In [9]:
model = cleanup_model(model, False)

In [10]:
model_prune_clean = "10_prune_clean.onnx"
model.save(model_prune_clean)

In [11]:
showInNetron(model_prune_clean)

Serving '10_prune_clean.onnx' at http://0.0.0.0:8083


In [12]:
model = model.transform(PruneChannels(prune_spec=layers_to_prune))

/home/gmoreno/uav/finn/deps/qonnx/src/qonnx/transformation/pruning.py:174: UserWarning: Can't propagate sparsity mask through op_type AveragePool
  warnings.warn("Can't propagate sparsity mask through op_type %s" % node.op_type)
/home/gmoreno/uav/finn/deps/qonnx/src/qonnx/transformation/pruning.py:174: UserWarning: Can't propagate sparsity mask through op_type Trunc
  warnings.warn("Can't propagate sparsity mask through op_type %s" % node.op_type)
/home/gmoreno/uav/finn/deps/qonnx/src/qonnx/transformation/pruning.py:174: UserWarning: Can't propagate sparsity mask through op_type Reshape
  warnings.warn("Can't propagate sparsity mask through op_type %s" % node.op_type)
/home/gmoreno/uav/finn/deps/qonnx/src/qonnx/transformation/pruning.py:174: UserWarning: Can't propagate sparsity mask through op_type Gemm
  warnings.warn("Can't propagate sparsity mask through op_type %s" % node.op_type)
/home/gmoreno/uav/finn/deps/qonnx/src/qonnx/transformation/pruning.py:174: UserWarning: Can't propaga

IndexError: index 17 is out of bounds for axis 1 with size 3